## Add SB1 geographic information.

In [1]:
import _utils
import numpy as np
import pandas as pd
import geopandas as gpd
from calitp.sql import to_snakecase

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import fsspec
from calitp import *
from calitp.storage import get_fs
fs = get_fs()
import os

In [3]:
import dask_geopandas as dg
import pyogrio

In [4]:
pd.options.display.max_columns = 200
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Non SHOPP-ATP-TIRCP

In [5]:
# Read in 10 Year non SHOPP with ATP and TIRCP
df = to_snakecase(pd.read_excel(f"{_utils.GCS_FILE_PATH}cleaned_data_atp_tircp.xlsx"))

In [6]:
# Subset to join.
non_shopp_subset = [
    "ppno",
    "project_name",
    "lead_agency",
    "previous_caltrans_nominations",
    "full_county_name",
    "district",
    "project_description",
    "current_phase",
    "primary_mode",
    "urban_rural",
    "total_project_cost__$1,000",
    "total_unfunded_need__$1,000",
    "notes",
    "shs_capacity_increase_detail",
    "current_phase",
]

In [7]:
df = df[non_shopp_subset]

In [8]:
df.district = df.district.map("{:02}".format)

### SB1 CSV

In [9]:
# Read in SB1
sb1 = to_snakecase(pd.read_csv(f"{_utils.GCS_FILE_PATH}RebuildingCA_map_Data.csv"))

In [10]:
sb1_subset = [
    "project_name",
    "implementing_agency",
    "sb1_program",
    "fiscal_year",
    "project_description",
    "total_cost",
    "sb1_funds",
    "is_sb1?",
    "project_status",
    "assembly_districts",
    "senate_districts",
    "congressional_districts",
    "counties",
    "cities",
    "caltrans_districts",
    "on_shs?",
]

In [11]:
sb1_2 = sb1[sb1_subset]

In [12]:
sb1.sb1_program.value_counts()

Local Streets and Roads                              3345
State Hwy Operations & Protection Program Major      2771
Highway Maintenance                                  1002
Active Transportation Program                         923
State of Good Repair                                  536
State Transit Assistance                              196
State Transportation Improvement Program              188
State Hwy Operations & Protection Program Minor A     169
Local Partnership Program - Formulaic                 135
Transit and Intercity Rail Capital Program             96
Trade Corridor Enhancement Program                     66
Local Partnership Program - Competitive                62
State Rail Assistance                                  53
Field Maintenance                                      48
Solutions for Congested Corridors Program              42
Name: sb1_program, dtype: int64

In [13]:
sb1_2.shape

(9632, 16)

In [14]:
# Fill in NA
sb1_2 = sb1_2.fillna(sb1_2.dtypes.replace({"float64": 0.0, "object": "None"}))

In [15]:
# Align funding
sb1_2 = _utils.align_funding_numbers(sb1_2, ["total_cost", "sb1_funds",])

#### Merge on Project Names

In [16]:
# Lower case and clean project names
for i in [sb1_2, df]:
    i['project_name'] = i['project_name'].str.lower().str.strip().str.split("20").str[0]

In [17]:
# 67 without accounting for districts
pd.merge(df, sb1_2, how="outer", on=["project_name"], indicator=True)[["_merge"]].value_counts()

_merge    
right_only    9563
left_only      840
both            69
dtype: int64

In [20]:
# 62 matches
pd.merge(df, sb1_2, how="outer", left_on=["project_name","district"], right_on = ["project_name", "caltrans_districts"], indicator=True)[["_merge"]].value_counts()

_merge    
right_only    9569
left_only      845
both            63
dtype: int64

In [21]:
project_title_m = pd.merge(df, sb1_2, how="left", left_on=["project_name","district"], right_on = ["project_name", "caltrans_districts"], indicator=True)

In [22]:
preview =['project_name','district','caltrans_districts', 'counties','full_county_name','project_description_x','project_description_y', "previous_caltrans_nominations",
                "sb1_program", "total_project_cost__$1,000", "total_cost"]

In [23]:
# project_title_m.loc[project_title_m._merge == 'both'][preview]

In [ ]:
project_title_m.sb1_program.value_counts()

Active Transportation Program                52
Local Partnership Program - Formulaic         3
Local Partnership Program - Competitive       2
Trade Corridor Enhancement Program            2
Solutions for Congested Corridors Program     2
State Transportation Improvement Program      2
Name: sb1_program, dtype: int64

### Tircp
* None of the projects from TIRCP are mapping, even though the names appear the same.

In [25]:
# sb1_tircp = sb1_2.loc[sb1_2.sb1_program == "Transit and Intercity Rail Capital Program"].reset_index(drop = True)

In [26]:
# sb1_tircp[['project_name','caltrans_districts','counties']].sort_values('project_name').head(2)

In [35]:
tircp_sb = sb1_2.loc[sb1_2.sb1_program.str.contains('Transit and Intercity')].reset_index(drop = True)

In [42]:
# tircp_sb[['project_name']].sort_values(by = 'project_name')

In [43]:
# tircp_shopp[['project_name']].sort_values(by = 'project_name')

In [36]:
tircp_shopp = df.loc[df.previous_caltrans_nominations.str.contains('TIRCP')].reset_index(drop = True)

In [37]:
# why are there no matches??
pd.merge(tircp_shopp, tircp_sb, how="outer", on=["project_name"], indicator=True)[["_merge"]].value_counts()

_merge    
right_only    95
left_only     93
both           1
dtype: int64

### SB1 Geojson
* https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA/RCA_Projects_032022/FeatureServer
* https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA/RCA_Projects_032022/FeatureServer/query
* https://medium.com/@thomas.deboer/how-to-download-from-an-arcgis-rest-server-using-python-df900db3e364

In [101]:
import requests
import urllib.parse

In [102]:
url = r"https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA/RCA_Projects_032022/FeatureServer/query"

In [103]:
#params = {
#    'geometry': '-118.21637221791077, 34.094916196179504',
#    'geometryType': 'esriGeometryPoint',
#    'returnGeometry': 'true', 
#    'f': 'pjson'
# }

In [104]:
# url_final = url + urllib.parse.urlencode(params)

In [105]:
# response = requests.get(url=url_final)

In [106]:
# data = response.text

In [108]:
# test = gpd.read_file(data)

In [110]:
# https://services.arcgisonline.com/arcgis/rest/services/Elevation/World_Hillshade/MapServer?f=json

In [109]:
# https://services.arcgisonline.com/arcgis/rest/services/Elevation/World_Hillshade/MapServer/tilemap/0/0/0/8/8?f=json